# Imports

In [1]:
import numpy as np
from Ising_Model_Fast import *

# Simulation

## Configuration

In [2]:
SEED = 42
np.random.seed(SEED)

N = 100
J1 = 0.5
J2 = 1.0
T = 0.5
MC_steps = int(1e6)

## Initialize net

In [3]:
# Generate a random 2D lattice

# Probability of value 1 [-1 has (p-1) probability]
p = 0.5
lattice = np.random.choice([1, -1], size=(N, N), p=[p, 1-p])

p_1 = 0.75
lattice_p = np.random.choice([1, -1], size=(N, N), p=[p_1, 1-p_1])

p_2 = 0.25
lattice_n = np.random.choice([1, -1], size=(N, N), p=[p_2, 1-p_2])

## Simulation parameters

In [4]:
# Get initial energy of the lattice
energy = get_energy(lattice, N, J1, J2)

# Desired number of states to save
n_images = 100

# Generate the appropriate spacing for the images
# Using logspace to get more images at the early stages (more change) and less at the end (less change)
img_spacing = set(np.logspace(0, np.log10(MC_steps), num=n_images, endpoint=True, dtype=int))

In [5]:
simulation_params = {
    'lattice' : lattice,
    'MC_steps' : MC_steps,
    'T' : T,
    'energy' : energy,
    'N' : N,
    'J1' : J1,
    'J2' : J2,
    'save_images' : True,
    'image_spacing' : img_spacing,
    'verbose' : 0
    }

# Run the Metropolis algorithm
spins, energies, images = metropolis(**simulation_params)

## Visualize results

In [7]:
paths = path_configuration(N, T)

# Save gif
create_gif(images, figures_dir=paths['figures'], filename='ising.gif', scale=5, fps=15, cmap='plasma')

# Save data
save_magnetization_data(spins, figures_dir=paths['data'])
save_energy_data(energies, figures_dir=paths['data'])
save_lattice_data(spins, figures_dir=paths['data'])

GIF saved as Simulation_N100_T0.5\figures\ising.gif


NameError: name 'save_magnetization_data' is not defined

In [6]:
n_Bs = 20
Bs = np.arange(0.1, 2, 0.05)

temperatures = 1 / Bs
T_steps = len(temperatures)
simulation_params['save_images'] = False
spins_T = np.empty((T_steps, MC_steps), dtype=np.int8)
energies_T = np.empty((T_steps, MC_steps), dtype=np.float32)
images_T = np.empty((T_steps, n_images-2, N, N), dtype=np.int8)

for i, T in enumerate(temperatures):
    print(f"Running T = {T:.2f}...")
    print("-" * 20)
    simulation_params['T'] = T
    spins_T[i], energies_T[i], _ = metropolis(**simulation_params)


Running T = 10.00...
--------------------
Running T = 6.67...
--------------------
Running T = 5.00...
--------------------
Running T = 4.00...
--------------------
Running T = 3.33...
--------------------
Running T = 2.86...
--------------------
Running T = 2.50...
--------------------
Running T = 2.22...
--------------------
Running T = 2.00...
--------------------
Running T = 1.82...
--------------------
Running T = 1.67...
--------------------
Running T = 1.54...
--------------------
Running T = 1.43...
--------------------
Running T = 1.33...
--------------------
Running T = 1.25...
--------------------
Running T = 1.18...
--------------------
Running T = 1.11...
--------------------
Running T = 1.05...
--------------------
Running T = 1.00...
--------------------
Running T = 0.95...
--------------------
Running T = 0.91...
--------------------
Running T = 0.87...
--------------------
Running T = 0.83...
--------------------
Running T = 0.80...
--------------------
Running T = 0.7